In [1]:
#script for generating a metasurface gds layout
#Splitting tasks in z sub tasks to handle storage constraints 

#The target GDS layout here is a phase profile conformal to a 3D curved curface 
#The goal of the meatsurace is to achieve 3D beam steering of an incoming pencil beam


for z in range(10):
    d = dir()
    for obj in d:
        #checking for built-in variables/functions
        if not obj.startswith('__') and obj!='z':
            #deleting the said obj, since a user-defined function
            del globals()[obj]
            
    import os

    # Clearing the Screen
    os.system('clear')
    !pip install gdspy
    !pip install mlxtend

    #import necessary libraries and frameworks
    import gdspy
    import pandas as pd
    import numpy as np
    import sys
    import math
    import matplotlib.pylab as plt
    import matplotlib.pyplot as mplt
    
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
    from matplotlib import cm
    from matplotlib.ticker import LinearLocator, FormatStrFormatter
    from numpy import savetxt
    from scipy.integrate import quad
    from datetime import date
    import time
    from mlxtend.data import iris_data
    from mlxtend.plotting import scatterplotmatrix

    #input parameters
    baseUnit = 1 #layout scale(microns)
    m=baseUnit*1e6 
    nm = m*1e-9
    um = m*1e-6
    mm = m*1e-3
    inch=m*25.4e-3

    bias=0; #geometrical correction factor to account for the oversize of the electron beam
    def_coeff=0.05; #approximated deformation of the meatsurface during the thermoforming process
    a=(1/(1+def_coeff)); #approximate inverse deformation of the metasurface 
    period = int(a*210) #period along x-axis
    periodX = period*nm #period along x-axis
    periodY = period*nm #period along y-axis
    Xmax =0.5*5*mm #metasurface unit length
    Ymax =0.5*5*mm #metasurface unit width
    startX=0*mm;
    Nx = round(Xmax/periodX)-1 #number of meta-atoms along x-axis
    Ny = round(Ymax/periodY)-1 #number of meta-atoms along y-axis
    phaseStep = 45*(np.pi/180);#phase, in degrees
    n_phaseLevels=8;
    wave=632*nm;

    
    
    today = date.today()
    date = today.strftime("%Y%m%d")
    index=z;

    #incident angle in degrees of the incoming pencil beam
    theta_incident=55; 
    n_angles=0;
    
    #Library obtained from RCWA simulation sweeps, Full 2Pi phase coverage along one polarization and constant phase along other dimension
    L_x_library=[0.051,0.112,0.119,0.126,0.136,0.147,0.156,0.160];
    L_y_library=[0.131,0.090,0.088,0.086,0.083,0.080,0.079,0.081];
    
    #Let us correct for electron beam bias 
    L_x=L_x_library-bias*np.ones(8);
    L_y=L_y_library-bias*np.ones(8);
    
    outputFileName = 'Filename_YRef0mm_Xref' +str(2.5*index)+'mm_AngleCorr_'+ date
    print(outputFileName)

    phase_vec=np.round((np.pi/180)*np.linspace(0,360,n_phaseLevels+1),2);

    
    #Our target surface spans 1.4 in wide (along X) and 0.1 in along Y, corresponding to 15 times a sweep o 25mm
    #We want Reference (X0,Y0) to be (in INCHES) (0.7 in ,0.035 in)
    #reference is bottom right corner when wearing the glasses.
    x_ref_0=np.round(0.8*inch+(index*2.5*mm),2);
    #x_ref_0=(0.8*inch+(index*2.5*mm));
    print('x_ref_0');
    print(x_ref_0);
    #0.168inch is the center line half width is 2.5mm
    y_ref_0=np.round(0.070*inch+(1*2.5*mm),2);
    #y_ref_0=(0.070*inch);
    print('y_ref_0');
    print(y_ref_0);
    x = np.linspace(0,Nx*periodX,Nx)+x_ref_0;
    y = np.linspace(0,Ny*periodY,Ny)+y_ref_0;
    x = np.float32(x);
    y = np.float32(y);

    r=np.sqrt(x**2+y**2);
    r = np.float32(r);

    X,Y = np.meshgrid(x,y);
        
    #polynomial coefficients analytically reproducing the shapoe of the curved surface considered
    p_00 =1.821416065725666704e+04
    p_10 =1.962822819760822167e+05
    p_01 =4.268565011736274027e+04
    p_20 =-1.977884403188649958e+04
    p_11 =-2.497281531933452641e+02
    p_02 =2.593006607777213503e+03
    p_30 =-2.553134480645263466e+04
    p_21 =8.538523511195862739e+03
    p_12 =1.137217508964654371e+04
    p_03 =-1.447115586661446650e+04
    p_40 =4.116622745496720017e+04
    p_31 =3.220572009199862350e+03
    p_22 =-1.394884655255972211e+04
    p_13 =-4.995105718702808190e+03
    p_04 =1.108907932096051627e+04
    p_50 =-3.230286883220907839e+04
    p_41 =-8.501203359072098465e+03
    p_32 =1.823257301292182456e+04
    p_23 =-4.141693838096583931e+03
    p_14 =-1.320500357549427463e+03
    p_05 =4.583656624828950612e+02
    p_60 =1.379004954273075055e+04
    p_51 =2.309881272377749610e+03
    p_42 =-4.878631667264818134e+03
    p_33 =-1.719168131482881563e+03
    p_24 =1.224684843449487380e+04
    p_15 =-5.682241123326056368e+03
    p_06 =1.062145253817967614e+03
    p_70 =-2.563484498541333778e+03
    p_61 =-2.289286760777909251e+02
    p_52 =1.642283823781504680e+03
    p_43 =9.322823794844978238e+02
    p_34 =-5.785141517454376299e+03
    p_25 =4.829502525006731958e+03
    p_16 =-2.542736009320173707e+03
    p_07 =5.049578241649963957e+02      

  
    phaseMap0=np.zeros((np.size(x,0),np.size(y,0)));
    phaseMap=np.zeros((np.size(x,0),np.size(y,0)));
    
    for i in range(np.size(x,0)):
        for j in range(np.size(y,0)):
            #input is in um, make sure you convert to inches
            #This is the only place where inches should be considered
            x_0=x[i]/inch;
            y_0=y[j]/inch;
            phaseMap0[i,j]=p_00 + p_10*x_0 + p_01*y_0 + p_20*(x_0**2) + p_11*x_0*y_0 + p_02*(y_0**2) + p_30*(x_0**3) + p_21*(x_0**2)*y_0 + p_12*x_0*(y_0**2) + p_03*(y_0**3) + p_40*(x_0**4) + p_31*(x_0**3)*y_0 + p_22*(x_0**2)*(y_0**2)+ p_13*(x_0**1)*(y_0**3)+ p_04*(y_0**4)+ p_50*(x_0**5) + p_41*(x_0**4)*y_0 + p_32*(x_0**3)*(y_0**2)+ p_23*(x_0**2)*(y_0**3)+ p_14*(x_0)*(y_0**4)+ p_05*(y_0**5)+ p_60*(x_0**6) + p_51*(x_0**5)*y_0 + p_42*(x_0**4)*(y_0**2)+ p_33*(x_0**3)*(y_0**3)+ p_24*(x_0**2)*(y_0**4)+ p_15*(x_0)*(y_0**5)+ p_06*(y_0**6)+ p_70*(x_0**7) + p_61*(x_0**6)*y_0 + p_52*(x_0**5)*(y_0**2)+ p_43*(x_0**4)*(y_0**3)+ p_34*(x_0**3)*(y_0**4)+ p_25*(x_0**2)*(y_0**5)+ p_16*(x_0)*(y_0**6)+ p_07*(y_0**7);
            phaseMap[i,j]=np.mod(phaseMap0[i,j],2*np.pi);

    ###### generating spatial map of (x,y) cells with the total size of a phaseMap

    def spatialMapGenerator(periodX, periodY, phaseMap):
        spatialMap = [[[np.round(x_ref_0+periodX*x,2),np.round(y_ref_0+periodY*y,2)] for x in range(np.size(phaseMap,1))] for y in range(np.size(phaseMap,0))]
        return spatialMap

    spatialMap = spatialMapGenerator(periodX, periodY, phaseMap)
    
    #function to clean up cells
    def clearCell(cell):
        cell.remove_polygons(lambda pts,layer,datatype:True)

    #layer and datatype tags
    ld_Si = {"layer":1, "datatype":1}

    #creating a gdspy library
    gdspy.current_library = gdspy.GdsLibrary()
    lib = gdspy.GdsLibrary()
    

    #MAPPING PHASE TO META-ATOM RADIUS
    #phaseData obtained with Reticolo RCWA script
    
    size=[[L_x[d],L_y[d]] for d in range(8)]; 
    print('size');
    print(size);
    phaseToSize = [[i*phaseStep, [size[i][0]*um,size[i][1]*um]] for i in range(8)]
    metaatomSize = [phaseToSize[i][1] for i in range(8)]
    phaseValue = [phaseToSize[i][0] for i in range(8)]

    #GENERATING META-ATOM lIBRARY
    disks = []
    for i in range(len(metaatomSize)):
        disks.append(gdspy.Rectangle((-0.5*metaatomSize[i][0],-0.5*metaatomSize[i][1]), (0.5*metaatomSize[i][0],0.5*metaatomSize[i][1])))  
    
    MAcell = []
    for i in range(len(metaatomSize)):
        MAcell.append(lib.new_cell('metaatom-'+str(i+1)).add(disks[i]))

    #mapping metasurface phase map to cell library 
    def phaseToCellMap(center, phase, phaseValue, MAcell):
        for i in range(len(phaseValue)):
            if phase<phaseValue[i]+phaseStep/2 and phase>=phaseValue[i]-phaseStep/2:
                return gdspy.CellReference(MAcell[i], center)
            elif phase>max(phaseValue)+phaseStep/2:
                return gdspy.CellReference(MAcell[0], center)
    
    #function to generate 2D map of rectangles (metasurface)
    def metalensGenerator(phaseMap, phaseToCellMap, spatialMap):
        metasurface = []
        for x in range(len(spatialMap)):
            for y in range(len(spatialMap)):
                if spatialMap[x][y] is not 'x':
                    metasurface.append(phaseToCellMap(spatialMap[x][y], phaseMap[x][y], phaseValue, MAcell))
        return metasurface

    metalens = metalensGenerator(phaseMap, phaseToCellMap, spatialMap)

    Lens=gdspy.Cell("Lens-1")
    Lens.add(metalens)
    
    #adding alignment marks
    am_pos = [(-2*mm,-2*mm)]
    cross_length = 200*um
    cross_width = 20*um

    crosses = []
    for i in range(len(am_pos)):
        rect1 = gdspy.Rectangle((am_pos[i][0]+cross_length/2, am_pos[i][1]+cross_width/2), (am_pos[i][0]-cross_length/2, am_pos[i][1]-cross_width/2))
        rect2 = gdspy.Rectangle((am_pos[i][0]+cross_width/2, am_pos[i][1]+cross_length/2), (am_pos[i][0]-cross_width/2, am_pos[i][1]-cross_length/2))
        cross = gdspy.boolean(rect1, rect2, "or")
        crosses.append(cross)
    align_marks = gdspy.Cell('alignment-marks')
    align_marks.add(crosses)

    #writing gds file
    start_time = time.time()
    gdspy.current_library = gdspy.GdsLibrary()
    gdspy.current_library.add(Lens)
    #Lens.add(align_marks)
    gdspy.write_gds(outputFileName + '.gds')
    print("--- %s seconds ---" % (time.time() - start_time))

<>:570: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:570: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\Admin\AppData\Local\Temp\ipykernel_3268\2919791728.py:570: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if spatialMap[x][y] is not 'x':


Screen will now be cleared in 5 Seconds
a
0.9523809523809523
period
200
periodX
0.2
Nx
12499
Ny
12499
LargeScale_cSi_490nm_PVC_air_FullGoogles_10nA_bias14nm_continuousStandalone_YRef0mm_Xref25.0mm_AngleCorr_20240830
phase_vec
[0.   0.79 1.57 2.36 3.14 3.93 4.71 5.5  6.28]
x_ref_0
45320.0
y_ref_0
4278.0
size
[[0.051, 0.131], [0.112, 0.09], [0.119, 0.088], [0.126, 0.086], [0.136, 0.083], [0.147, 0.08], [0.156, 0.079], [0.16, 0.081]]


C:\Users\Admin\AppData\Local\Temp\ipykernel_3268\2919791728.py:570: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if spatialMap[x][y] is not 'x':
C:\Users\Admin\AppData\Local\Temp\ipykernel_3268\2919791728.py:570: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if spatialMap[x][y] is not 'x':


OSError: [Errno 28] No space left on device

In [ ]:
#fig=plt.figure();
#pos=plt.imshow(X)
#fig.colorbar(pos)
#print(x-x_ref_0);
#print(x_ref_0);
#print(Nx)
#print(periodX)
#print(Nx*periodX)
#x0 = np.linspace(0,(Nx*periodX),Nx);
#print(x0);
    

In [ ]:
print('phaseMap');
print(phaseMap);
print('x');
print(x);
print(x/inch);
print('y');
print(y);
print(y/inch);
fig=plt.figure();
plt.imshow(phaseMap);
plt.colorbar();